In [19]:
import pandas as pd
from nba_api.stats.endpoints import playbyplayv2, leaguegamelog
from datetime import datetime, timedelta
import time

## New PBP Data
Retrieves and processes the raw PBP data from the NBA API. 

Undergoes the following steps:
- adds columns for the game date, home and away teams
- adds a column for play clock in seconds (rather than a string)
- adds shifted columns (i.e. the values from the row above) for PBP analysis
- adds columns for whether possession changed on each play, and the current team of possession
- adds a column for the score change on each play

In [182]:
# Test - re-load data from CSV
# This version includes game_ID_data, but not the other additions
pbp = pd.read_csv('PBP_RAW_20250208.csv')
pbp = pbp.drop(['Unnamed: 0'], axis=1)
# Convert back to strings to match for merge
pbp.GAME_ID = pbp.GAME_ID.apply(lambda row: "00" + str(row))

In [188]:
def get_games_dict(last_n_days=7):
  
  date_from = datetime.today() - timedelta(days=last_n_days)
  game_log = leaguegamelog.LeagueGameLog(
    date_from_nullable=date_from.strftime('%m/%d/%Y')
  ).get_data_frames()[0]

  game_ID_list = list(set(game_log[game_log.MATCHUP.str.contains("@")]["GAME_ID"].to_list()))

  return {game_ID_list[i]: [game_log[game_log.GAME_ID == game_ID_list[i]].GAME_DATE.values[0],
                                game_log[game_log.GAME_ID == game_ID_list[i]].MATCHUP.values[0][0:3],
                                game_log[game_log.GAME_ID == game_ID_list[i]].MATCHUP.values[0][-4:]]
                                for i in range(len(game_ID_list))}

In [217]:
def get_games_pbp(df, games_dict):
  for game_id in games_dict.keys():
    print(f"Fetching game {game_id}...")
    game_pbp = playbyplayv2.PlayByPlayV2(game_id=game_id).get_data_frames()[0]
    df = pd.concat([df, game_pbp], ignore_index=True)

  return df

In [265]:
def pbp_shift_rows(df):
  df['PCTIME_SECONDS'] = df['PCTIMESTRING'].map(lambda x: int(x.split(":")[0])*60 + int(x.split(":")[1]))
  df = df.sort_values(['GAME_DATE', 'GAME_ID', 'PERIOD','PCTIME_SECONDS','EVENTNUM'], ascending=[True, True, True, False, True])
  df = df.reset_index(drop=True)

  #Update the SCORE column to fill in blanks
  df.loc[(df['EVENTMSGTYPE'] == 12) & (df['PERIOD'] == 1), 'SCORE'] = "0 - 0"
  df["SCORE"] = df["SCORE"].fillna(method="ffill")

  #New columns for possession formulae
  df['EVENTMSGTYPE_SHIFT'] = df['EVENTMSGTYPE'].shift(-1)
  df['EVENTMSGACTIONTYPE_SHFIT'] = df['EVENTMSGACTIONTYPE'].shift(-1)
  df['PCTIME_SECONDS_SHFIT'] = df['PCTIME_SECONDS'].shift(-1)
  df['PLAYER1_TEAM_ID_SHIFT'] = df['PLAYER1_TEAM_ID'].shift(-1)
  df['PLAYER1_ID_SHIFT'] = df['PLAYER1_ID'].shift(-1)
  df['PCTIME_SECONDS_SHIFT'] = df['PCTIME_SECONDS'].shift(-1)
  df['SCORE_SHIFT'] = df['SCORE'].shift(1)
  df.loc[(df['EVENTMSGTYPE'] == 12) & (df['PERIOD'] == 1), 'SCORE_SHIFT'] = "0 - 0"

  return df


# Possession end check functions
def poss_end_fg(row):
    return not (row['EVENTMSGTYPE_SHIFT'] == 6 and row['PCTIME_SECONDS'] == row['PCTIME_SECONDS_SHIFT'])

def poss_end_rebound(row):
    return row['PLAYER1_TEAM_ID'] != row['PLAYER1_TEAM_ID_SHIFT']

def poss_end_ft(row):
    return (row['EVENTMSGACTIONTYPE'] in {10, 12, 15} and 
            (row['EVENTMSGTYPE_SHIFT'] != 4 or poss_end_rebound(row)))

# map event types to their corresponding functions
POSS_OUTCOME_FUNCS = {
    1: poss_end_fg,
    2: poss_end_rebound,
    3: poss_end_ft
}

def poss_end_check(row):    
    return (POSS_OUTCOME_FUNCS.get(row['EVENTMSGTYPE'], 
                                   lambda _: row['EVENTMSGTYPE'] in {5, 13}))(row)



def pbp_possessions(df):

    df = df.copy()

    df['POSSESSION_END'] = df.apply(poss_end_check, axis=1)
    
    # get team names per game
    game_teams = df.groupby("GAME_ID").agg(
        home_team=("HOME_TEAM", "first"),
        away_team=("AWAY_TEAM", "first")
    ).to_dict(orient="index")

    # determine possession team
    possession_team = []
    current_game = None
    current_team = None

    for idx, row in df.iterrows():
        game_id = row["GAME_ID"]

        # if new game, reset teams and find first possession
        if game_id != current_game:
            current_game = game_id
            teams = (game_teams[game_id]["home_team"], game_teams[game_id]["away_team"])

            # check for first jump ball winner
            jump_ball_row = df[(df["GAME_ID"] == game_id) & (df["EVENTMSGTYPE"] == 10)].head(1)

            if not jump_ball_row.empty:
                current_team = jump_ball_row.iloc[0]["PLAYER3_TEAM_ABBREVIATION"]
            else:
                # fallback: find first violation (EVENTMSGTYPE == 7) i.e. jump ball violation
                violation_row = df[(df["GAME_ID"] == game_id) & (df["EVENTMSGTYPE"] == 7)].head(1)

                if not violation_row.empty:
                    violating_team = violation_row.iloc[0]["PLAYER1_TEAM_ABBREVIATION"]
                    # possession goes to the team that didn't commit the violation
                    current_team = teams[1] if violating_team == teams[0] else teams[0]
                else:
                    raise ValueError(f"No valid first possession found for GAME_ID {game_id}")

        possession_team.append(current_team)

        # switch possession if POSSESSION_END is True
        if row["POSSESSION_END"]:
            current_team = teams[1] if current_team == teams[0] else teams[0]

    df["POSSESSION_TEAM_ABBREVIATION"] = possession_team

    # vectorized score change calculation
    def event_score(row):
        start = row['SCORE_SHIFT'].split(" - ")
        end = row['SCORE'].split(" - ")
        return max(int(end[0]) - int(start[0]), int(end[1]) - int(start[1]))

    df['SCORE_CHANGE'] = df.apply(event_score, axis=1)
    
    return df

In [231]:
def process_pbp(df, games_dict={}, games_info=False):

  if games_info:
    gameID_df = pd.DataFrame.from_dict(games_dict, orient='index', columns=['GAME_DATE', 'AWAY_TEAM', 'HOME_TEAM'])
    gameID_df.index.name = 'GAME_ID'
    gameID_df.reset_index(inplace=True)

    df = df.merge(gameID_df, on='GAME_ID', how='left')

  df = pbp_shift_rows(df)
  df = pbp_possessions(df)

  return df

In [210]:
def get_new_pbp(last_n_days=7):

  games_dict = get_games_dict(last_n_days=last_n_days)

  games_df = get_games_pbp(pd.DataFrame(), games_dict)
  games_df = process_pbp(games_df, games_dict, games_info=True)

  return games_df

## Working Cells

In [235]:
pbp = pbp_possessions(pbp)

ValueError: No jump ball event found for GAME_ID 0022400495

In [240]:
check_game = playbyplayv2.PlayByPlayV2(game_id="0022400495").get_data_frames()[0]

In [ ]:

for game_id in gameID_df.GAME_ID:

  print(f"Fetching game {game_id}...")
  game_pbp = playbyplayv2.PlayByPlayV2(game_id=game_id).get_data_frames()[0]
  pbp = pd.concat([pbp, game_pbp], ignore_index=True)

In [82]:
pbp.to_csv("PBP_RAW_20250208.csv")

In [173]:
# Re-load CSV

pbp = pd.read_csv('PBP_RAW_20250208.csv')

gameID_df = pd.DataFrame.from_dict(game_ID_dict, orient='index', columns=['GAME_DATE', 'AWAY_TEAM', 'HOME_TEAM'])
gameID_df.index.name = 'GAME_ID'
gameID_df.reset_index(inplace=True)

pbp.GAME_ID = pbp.GAME_ID.apply(lambda row: "00" + str(row))
pbp = pbp.merge(gameID_df, on='GAME_ID', how='left')

In [104]:
pbp = pbp.drop(['Unnamed: 0'], axis=1)

Data Updates:

- fill score column values
- convert `PCTIMESTRING` to seconds
- add game date and teams
- 

In [120]:
pbp_small.head()

,GAME_ID,EVENTNUM,EVENTMSGTYPE,EVENTMSGACTIONTYPE,PERIOD,WCTIMESTRING,PCTIMESTRING,HOMEDESCRIPTION,NEUTRALDESCRIPTION,VISITORDESCRIPTION,...,AWAY_TEAM,HOME_TEAM,PCTIME_SECONDS,EVENTMSGTYPE_SHIFT,EVENTMSGACTIONTYPE_SHFIT,PCTIME_SECONDS_SHFIT,PLAYER1_TEAM_ID_SHIFT,PLAYER1_ID_SHIFT,PCTIME_SECONDS_SHIFT,SCORE_SHIFT
0,0022400061,2,12,0,1,7:36 PM,12:00,NaN,Start of 1st Period (7:36 PM EST),NaN,...,BOS,NYK,720,10.0,0.0,720.0,1.610613e+09,201143.0,720.0,0 - 0
1,0022400061,4,10,0,1,7:36 PM,12:00,Jump Ball Horford vs. Towns: Tip to White,NaN,NaN,...,BOS,NYK,720,1.0,79.0,708.0,1.610613e+09,1628369.0,708.0,0 - 0
2,0022400061,7,1,79,1,7:36 PM,11:48,Tatum 27' 3PT Pullup Jump Shot (3 PTS) (White ...,NaN,NaN,...,BOS,NYK,708,1.0,6.0,687.0,1.610613e+09,1628384.0,687.0,0 - 0
3,0022400061,9,1,6,1,7:37 PM,11:27,NaN,NaN,Anunoby 6' Driving Layup (2 PTS) (Brunson 1 AST),...,BOS,NYK,687,2.0,79.0,676.0,1.610613e+09,1628369.0,676.0,0 - 3
4,0022400061,11,2,79,1,7:37 PM,11:16,MISS Tatum 26' 3PT Pullup Jump Shot,NaN,NaN,...,BOS,NYK,676,4.0,0.0,675.0,1.610613e+09,1628384.0,675.0,2 - 3


In [171]:
poss_end_check(pbp_small.iloc[3])

True

In [128]:
pbp_small['POSSESSION_END'] = pbp_small.apply(poss_end_check, axis=1)

/var/folders/2p/l_6wwc653kn38c5ng_dn2pnm0000gn/T/ipykernel_41571/3503058728.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pbp_small['POSSESSION_END'] = pbp_small.apply(poss_end_check, axis=1)


In [139]:
pbp_small.head(10)

,GAME_ID,EVENTNUM,EVENTMSGTYPE,EVENTMSGACTIONTYPE,PERIOD,WCTIMESTRING,PCTIMESTRING,HOMEDESCRIPTION,NEUTRALDESCRIPTION,VISITORDESCRIPTION,...,HOME_TEAM,PCTIME_SECONDS,EVENTMSGTYPE_SHIFT,EVENTMSGACTIONTYPE_SHFIT,PCTIME_SECONDS_SHFIT,PLAYER1_TEAM_ID_SHIFT,PLAYER1_ID_SHIFT,PCTIME_SECONDS_SHIFT,SCORE_SHIFT,POSSESSION_END
0,0022400061,2,12,0,1,7:36 PM,12:00,NaN,Start of 1st Period (7:36 PM EST),NaN,...,NYK,720,10.0,0.0,720.0,1.610613e+09,201143.0,720.0,0 - 0,False
1,0022400061,4,10,0,1,7:36 PM,12:00,Jump Ball Horford vs. Towns: Tip to White,NaN,NaN,...,NYK,720,1.0,79.0,708.0,1.610613e+09,1628369.0,708.0,0 - 0,False
2,0022400061,7,1,79,1,7:36 PM,11:48,Tatum 27' 3PT Pullup Jump Shot (3 PTS) (White ...,NaN,NaN,...,NYK,708,1.0,6.0,687.0,1.610613e+09,1628384.0,687.0,0 - 0,True
3,0022400061,9,1,6,1,7:37 PM,11:27,NaN,NaN,Anunoby 6' Driving Layup (2 PTS) (Brunson 1 AST),...,NYK,687,2.0,79.0,676.0,1.610613e+09,1628369.0,676.0,0 - 3,True
4,0022400061,11,2,79,1,7:37 PM,11:16,MISS Tatum 26' 3PT Pullup Jump Shot,NaN,NaN,...,NYK,676,4.0,0.0,675.0,1.610613e+09,1628384.0,675.0,2 - 3,True
5,0022400061,12,4,0,1,7:37 PM,11:15,NaN,NaN,Anunoby REBOUND (Off:0 Def:1),...,NYK,675,1.0,41.0,669.0,1.610613e+09,1628973.0,669.0,2 - 3,False
6,0022400061,13,1,41,1,7:37 PM,11:09,NaN,NaN,Brunson 3' Running Layup (2 PTS),...,NYK,669,1.0,101.0,654.0,1.610613e+09,1628401.0,654.0,2 - 3,True
7,0022400061,14,1,101,1,7:37 PM,10:54,White 5' Driving Floating Jump Shot (2 PTS) (T...,NaN,NaN,...,NYK,654,1.0,79.0,635.0,1.610613e+09,1628973.0,635.0,4 - 3,True
8,0022400061,16,1,79,1,7:38 PM,10:35,NaN,NaN,Brunson 13' Pullup Jump Shot (4 PTS),...,NYK,635,1.0,1.0,621.0,1.610613e+09,201143.0,621.0,4 - 5,True
9,0022400061,17,1,1,1,7:38 PM,10:21,Horford 25' 3PT Jump Shot (3 PTS) (Tatum 2 AST),NaN,NaN,...,NYK,621,1.0,58.0,593.0,1.610613e+09,1626157.0,593.0,6 - 5,True


In [166]:
def pbp_new_rows2(df):

    df = df.copy()
    
    # get team names per game
    game_teams = df.groupby("GAME_ID").agg(
        home_team=("HOME_TEAM", "first"),
        away_team=("AWAY_TEAM", "first")
    ).to_dict(orient="index")

    # determine possession team
    possession_team = []
    current_game = None
    current_team = None

    for idx, row in df.iterrows():
        game_id = row["GAME_ID"]

        # if new game, reset teams and find first jump ball
        if game_id != current_game:
            current_game = game_id
            teams = (game_teams[game_id]["home_team"], game_teams[game_id]["away_team"])
            
            jump_ball_idx = df[(df["GAME_ID"] == game_id) & (df["EVENTMSGTYPE"] == 10)].index.min()
            if pd.isna(jump_ball_idx):
                raise ValueError(f"No jump ball event found for GAME_ID {game_id}")

            current_team = df.at[jump_ball_idx, "PLAYER3_TEAM_ABBREVIATION"]

        possession_team.append(current_team)

        # switch possession if POSSESSION_END is True
        if row["POSSESSION_END"]:
            current_team = teams[1] if current_team == teams[0] else teams[0]

    df["POSSESSION_TEAM_ABBREVIATION"] = possession_team

    # vectorized score change calculation
    def event_score(row):
        start = row['SCORE_SHIFT'].split(" - ")
        end = row['SCORE'].split(" - ")
        return max(int(end[0]) - int(start[0]), int(end[1]) - int(start[1]))

    df['SCORE_CHANGE'] = df.apply(event_score, axis=1)
    
    return df

In [167]:
pbp_new_rows2(pbp_small)

,GAME_ID,EVENTNUM,EVENTMSGTYPE,EVENTMSGACTIONTYPE,PERIOD,WCTIMESTRING,PCTIMESTRING,HOMEDESCRIPTION,NEUTRALDESCRIPTION,VISITORDESCRIPTION,...,EVENTMSGTYPE_SHIFT,EVENTMSGACTIONTYPE_SHFIT,PCTIME_SECONDS_SHFIT,PLAYER1_TEAM_ID_SHIFT,PLAYER1_ID_SHIFT,PCTIME_SECONDS_SHIFT,SCORE_SHIFT,POSSESSION_END,POSSESSION_TEAM_ABBREVIATION,SCORE_CHANGE
0,0022400061,2,12,0,1,7:36 PM,12:00,NaN,Start of 1st Period (7:36 PM EST),NaN,...,10.0,0.0,720.0,1.610613e+09,201143.0,720.0,0 - 0,False,BOS,0
1,0022400061,4,10,0,1,7:36 PM,12:00,Jump Ball Horford vs. Towns: Tip to White,NaN,NaN,...,1.0,79.0,708.0,1.610613e+09,1628369.0,708.0,0 - 0,False,BOS,0
2,0022400061,7,1,79,1,7:36 PM,11:48,Tatum 27' 3PT Pullup Jump Shot (3 PTS) (White ...,NaN,NaN,...,1.0,6.0,687.0,1.610613e+09,1628384.0,687.0,0 - 0,True,BOS,3
3,0022400061,9,1,6,1,7:37 PM,11:27,NaN,NaN,Anunoby 6' Driving Layup (2 PTS) (Brunson 1 AST),...,2.0,79.0,676.0,1.610613e+09,1628369.0,676.0,0 - 3,True,NYK,2
4,0022400061,11,2,79,1,7:37 PM,11:16,MISS Tatum 26' 3PT Pullup Jump Shot,NaN,NaN,...,4.0,0.0,675.0,1.610613e+09,1628384.0,675.0,2 - 3,True,BOS,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0022400063,193,4,0,2,7:44 PM,11:24,NaN,NaN,Mathurin REBOUND (Off:0 Def:2),...,10.0,0.0,677.0,1.610613e+09,1641842.0,677.0,28 - 31,False,DET,0
996,0022400063,194,10,0,2,7:45 PM,11:17,,NaN,NaN,...,5.0,2.0,674.0,1.610613e+09,1631097.0,674.0,28 - 31,False,DET,0
997,0022400063,197,5,2,2,7:45 PM,11:14,Holland II STEAL (2 STL),NaN,Mathurin Lost Ball Turnover (P2.T7),...,6.0,1.0,668.0,1.610613e+09,1630167.0,668.0,28 - 31,True,DET,0
998,0022400063,200,6,1,2,7:46 PM,11:08,NaN,NaN,Toppin P.FOUL (P1.T1) (T.Ricks),...,8.0,0.0,668.0,1.610613e+09,1630191.0,668.0,28 - 31,False,IND,0


In [114]:
pbp = pbp_NewRows(pbp, game_info=True)

In [118]:
pbp.head()

,GAME_ID,EVENTNUM,EVENTMSGTYPE,EVENTMSGACTIONTYPE,PERIOD,WCTIMESTRING,PCTIMESTRING,HOMEDESCRIPTION,NEUTRALDESCRIPTION,VISITORDESCRIPTION,...,AWAY_TEAM,HOME_TEAM,PCTIME_SECONDS,EVENTMSGTYPE_SHIFT,EVENTMSGACTIONTYPE_SHFIT,PCTIME_SECONDS_SHFIT,PLAYER1_TEAM_ID_SHIFT,PLAYER1_ID_SHIFT,PCTIME_SECONDS_SHIFT,SCORE_SHIFT
0,0022400061,2,12,0,1,7:36 PM,12:00,NaN,Start of 1st Period (7:36 PM EST),NaN,...,BOS,NYK,720,10.0,0.0,720.0,1.610613e+09,201143.0,720.0,0 - 0
1,0022400061,4,10,0,1,7:36 PM,12:00,Jump Ball Horford vs. Towns: Tip to White,NaN,NaN,...,BOS,NYK,720,1.0,79.0,708.0,1.610613e+09,1628369.0,708.0,0 - 0
2,0022400061,7,1,79,1,7:36 PM,11:48,Tatum 27' 3PT Pullup Jump Shot (3 PTS) (White ...,NaN,NaN,...,BOS,NYK,708,1.0,6.0,687.0,1.610613e+09,1628384.0,687.0,0 - 0
3,0022400061,9,1,6,1,7:37 PM,11:27,NaN,NaN,Anunoby 6' Driving Layup (2 PTS) (Brunson 1 AST),...,BOS,NYK,687,2.0,79.0,676.0,1.610613e+09,1628369.0,676.0,0 - 3
4,0022400061,11,2,79,1,7:37 PM,11:16,MISS Tatum 26' 3PT Pullup Jump Shot,NaN,NaN,...,BOS,NYK,676,4.0,0.0,675.0,1.610613e+09,1628384.0,675.0,2 - 3


In [ ]:
db_path = "pbp25.csv" ## CHANGE TO CORRECT

def get_last_week_games():
    today = datetime.today()
    last_week = today - timedelta(days=7)
    return last_week.strftime('%m-%d-%Y'), today.strftime('%m-%d-%Y')

def fetch_play_by_play(game_id):
    time.sleep(0.6)  # avoid rate limiting
    pbp = playbyplayv2.PlayByPlayV2(game_id=game_id).get_data_frames()[0]
    return pbp

def fetch_games_and_update_db():
    start_date, end_date = get_last_week_games()
    game_log = leaguegamelog.LeagueGameLog(date_from_nullable=start_date, date_to_nullable=end_date).get_data_frames()[0]
    game_ids = game_log["GAME_ID"].tolist()
    
    try:
        pbp_data = pd.read_csv(db_path)
    except FileNotFoundError:
        pbp_data = pd.DataFrame()
    
    for game_id in game_ids:
        print(f"Fetching game {game_id}...")
        game_pbp = fetch_play_by_play(game_id)
        pbp_data = pd.concat([pbp_data, game_pbp], ignore_index=True)
    
    pbp_data.to_csv(db_path, index=False)

In [ ]:
def analyze_self_rebounded_ft():
    pbp_data = pd.read_csv(db_path)
    
    missed_fts = pbp_data[(pbp_data["EVENTMSGTYPE"] == 4) & (pbp_data["PLAYER1_ID"].notnull())]
    results = []
    
    for _, row in missed_fts.iterrows():
        player_id = row["PLAYER1_ID"]
        player_name = row["PLAYER1_NAME"]
        game_id = row["GAME_ID"]
        eventnum = row["EVENTNUM"]
        
        next_events = pbp_data[(pbp_data["GAME_ID"] == game_id) & (pbp_data["EVENTNUM"] > eventnum) & (pbp_data["EVENTNUM"] <= eventnum + 3)]
        scored_event = next_events[next_events["EVENTMSGTYPE"].isin([1, 3])]
        
        if not scored_event.empty:
            points = 2 if 3 in scored_event["EVENTMSGTYPE"].values else 3
            results.append((player_id, player_name, points))
    
    results_df = pd.DataFrame(results, columns=["Player ID", "Player Name", "Points"])
    summary_df = results_df.groupby(["Player ID", "Player Name"]).agg(
        FTA_Self_Rebounded=("Points", "count"),
        Total_Points=("Points", "sum"),
        PPP=("Points", "mean")
    ).reset_index()
    
    summary_df = summary_df.sort_values(by="Total_Points", ascending=False)
    return summary_df

if __name__ == "__main__":
    fetch_games_and_update_db()
    result_df = analyze_self_rebounded_ft()
    # import ace_tools as tools
    # tools.display_dataframe_to_user("League Leaders in Self-Rebounded FT Points", result_df)
